In [1]:
import json
import time  # <--- Importação necessária
import threading
import ollama


MODEL_NAME_SCHEMA = "SCHEMA_V3"

# Controle para acesso serializado + warm-up
_schema_llm_lock = threading.Lock()
_schema_llm_warmed_up = False

def _warm_up_schema_llm():
    """Garante que o modelo esteja carregado na VRAM."""
    global _schema_llm_warmed_up
    if _schema_llm_warmed_up:
        return
    try:
        # Chamada rápida para load
        ollama.chat(model=MODEL_NAME_SCHEMA, messages=[{"role": "user", "content": "ping"}])
        _schema_llm_warmed_up = True
    except Exception as e:
        print(f"[WARN] Falha no warm-up {MODEL_NAME_SCHEMA}: {e}")

def get_ai_schema_dict(question: str) -> dict:
    """
    Invoca a IA Schema forçando saída JSON nativa do Ollama.
    Retorna o dict do schema com uma chave extra '_generation_time'.
    """
    with _schema_llm_lock:
        _warm_up_schema_llm()
        
        # 1. Inicia cronômetro (apenas para a geração)
        start_time = time.time()
        
        try:
            resp = ollama.chat(
                model=MODEL_NAME_SCHEMA,
                messages=[{"role": "user", "content": question}],
                format="json",  # Força saída JSON estruturada
                options={
                    "temperature": 0.0, # Determinístico
                    "num_ctx": 8192     # Contexto aumentado para suportar System Prompt maior
                },
                stream=False
            )
            
            # 2. Para cronômetro
            end_time = time.time()
            duration = end_time - start_time
            
            # Processa o JSON
            content = resp["message"]["content"]
            data = json.loads(content)
            
            # 3. Injeta o tempo no resultado para o Pipeline ler depois
            data["_generation_time"] = duration
            
            return data
            
        except json.JSONDecodeError:
            # Fallback se o JSON quebrar
            print(f"[ERRO] Falha ao decodificar JSON. Retorno bruto: {content}")
            return {
                "intent": "lookup", 
                "needs_query": False, 
                "reason": "Erro na geração do schema JSON.",
                "_generation_time": time.time() - start_time
            }
        except Exception as e:
            print(f"[ERRO] Erro na chamada Ollama: {e}")
            # Retorna dicionário vazio mas com o tempo que gastou até o erro
            return {"_generation_time": time.time() - start_time}

In [2]:
# =========================
# CÉLULA 2 — RAG VETORIAL + PIPELINE COMPLETO (IA1 + IA2 + NEO4J)
# =========================

import json
import os
import time
import re
from pathlib import Path
import threading
from collections import defaultdict

import numpy as np
import ollama
from neo4j import GraphDatabase

# --------------------------------------------------------
# 0. CONFIG
# --------------------------------------------------------
INDEX_DIR = Path("schema_vetorizado")
EMB_PATH = INDEX_DIR / "schema_emb.npy"
CHUNKS_PATH = INDEX_DIR / "schema_chunks.json"

EMBED_MODEL_NAME = os.environ.get("EMBED_MODEL_NAME", "mxbai-embed-large")
MODEL_NAME_CYPHER = "CYPHER_V2"

NEO4J_URI = os.environ.get("NEO4J_URI", "neo4j://127.0.0.1:7687")
NEO4J_USER = os.environ.get("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.environ.get("NEO4J_PASSWORD", "zWF$yls*J;K:DtC3")
NEO4J_DATABASE = os.environ.get("NEO4J_DATABASE", "vectoria")

# --------------------------------------------------------
# 1. ESTADO GLOBAL DO SCHEMA
# --------------------------------------------------------
_SCHEMA_CHUNKS = []
_SCHEMA_EMB_MATRIX = None
_SCHEMA_INDEX_LOADED = False

# Índices determinísticos (NOVO)
_SCHEMA_RELS = []
_SCHEMA_ENTITY_DESCR = {}
_SCHEMA_FIELDS_BY_ENTITY = defaultdict(list)
_SCHEMA_TIPOS_BY_ENTITY = defaultdict(list)
_SCHEMA_IDX_KIND_ENTITY = defaultdict(list)  # (kind, entity) -> [idxs]

_cypher_lock = threading.Lock()
_cypher_warmed_up = False


# --------------------------------------------------------
# 1.1 UTILITÁRIOS
# --------------------------------------------------------
def _truncate(text, max_len=150):
    if not text:
        return ""
    text = str(text).strip().replace("\n", " ")
    return text if len(text) <= max_len else text[: max_len - 3] + "..."


# --------------------------------------------------------
# 1.2 CARGA DO ÍNDICE + CONSTRUÇÃO DE MAPAS (NOVO)
# --------------------------------------------------------
def _build_schema_maps():
    """Cria índices para acesso determinístico por entidade/kind."""
    global _SCHEMA_RELS, _SCHEMA_ENTITY_DESCR, _SCHEMA_FIELDS_BY_ENTITY
    global _SCHEMA_TIPOS_BY_ENTITY, _SCHEMA_IDX_KIND_ENTITY

    _SCHEMA_RELS = []
    _SCHEMA_ENTITY_DESCR = {}
    _SCHEMA_FIELDS_BY_ENTITY = defaultdict(list)
    _SCHEMA_TIPOS_BY_ENTITY = defaultdict(list)
    _SCHEMA_IDX_KIND_ENTITY = defaultdict(list)

    for i, ch in enumerate(_SCHEMA_CHUNKS):
        ent = ch.get("entity")
        kind = ch.get("kind")

        _SCHEMA_IDX_KIND_ENTITY[(kind, ent)].append(i)

        if kind == "entity":
            _SCHEMA_ENTITY_DESCR.setdefault(ent, ch.get("descr", ""))
        elif kind == "field":
            _SCHEMA_FIELDS_BY_ENTITY[ent].append(ch)
        elif kind == "tipo":
            _SCHEMA_TIPOS_BY_ENTITY[ent].append(ch)
        elif kind == "rel":
            _SCHEMA_RELS.append(ch)


def _load_schema_embedding_index() -> bool:
    global _SCHEMA_CHUNKS, _SCHEMA_EMB_MATRIX, _SCHEMA_INDEX_LOADED

    if not EMB_PATH.exists() or not CHUNKS_PATH.exists():
        return False

    try:
        _SCHEMA_EMB_MATRIX = np.load(EMB_PATH)
        with CHUNKS_PATH.open(encoding="utf-8") as f:
            _SCHEMA_CHUNKS = json.load(f)

        # NOVO: constrói mapas determinísticos
        _build_schema_maps()

        _SCHEMA_INDEX_LOADED = True
        return True
    except:
        return False


def _ensure_schema_index_loaded():
    if not _SCHEMA_INDEX_LOADED:
        _load_schema_embedding_index()


# --------------------------------------------------------
# 1.3 EMBEDDING + BUSCA VETORIAL (mantida)
# --------------------------------------------------------
def _embed_text(text):
    return ollama.embeddings(model=EMBED_MODEL_NAME, prompt=text)["embedding"]


def search_schema_chunks(query, allowed_entities=None, top_k=50):
    _ensure_schema_index_loaded()
    if _SCHEMA_EMB_MATRIX is None:
        return []

    q_emb = np.array(_embed_text(query), dtype="float32")
    q_emb /= (np.linalg.norm(q_emb) + 1e-9)

    emb_norm = _SCHEMA_EMB_MATRIX / (
        np.linalg.norm(_SCHEMA_EMB_MATRIX, axis=1, keepdims=True) + 1e-9
    )
    sims = emb_norm @ q_emb

    idxs = np.arange(len(_SCHEMA_CHUNKS))
    if allowed_entities:
        allowed = set(allowed_entities)
        mask = np.array([c.get("entity") in allowed for c in _SCHEMA_CHUNKS])
        if np.any(mask):
            idxs = idxs[mask]
            sims = sims[mask]

    if sims.size == 0:
        return []

    k_real = min(top_k, sims.shape[0])
    top_idx = np.argpartition(-sims, k_real - 1)[:k_real]
    sorted_idx = top_idx[np.argsort(-sims[top_idx])]

    return [_SCHEMA_CHUNKS[int(idxs[i])] for i in sorted_idx]


def extract_id_from_name(name):
    if not name:
        return None
    match = re.search(r"(\d+)$", str(name).strip())
    if match:
        return match.group(1)
    return None


# --------------------------------------------------------
# 1.4 RELAÇÕES ENTRE ENTIDADES DO PLANO (NOVO)
# --------------------------------------------------------
def get_relations_among_entities(target_entities):
    """Retorna todas as relações cujo (de, para) estão dentro do conjunto alvo."""
    _ensure_schema_index_loaded()

    ents = set(target_entities or [])
    if not ents:
        return []

    seen = set()
    rels = []

    for ch in _SCHEMA_RELS:
        de = ch.get("de")
        para = ch.get("para")
        tipo_rel = ch.get("tipo_rel")

        if de in ents and para in ents and tipo_rel:
            key = (de, tipo_rel, para)
            if key not in seen:
                seen.add(key)
                rels.append(ch)

    return rels


# --------------------------------------------------------
# 1.5 RANK DOS TIPOS POR SIMILARIDADE (NOVO)
# --------------------------------------------------------
def rank_tipos(entity, query, top_k=5):
    _ensure_schema_index_loaded()
    idxs = _SCHEMA_IDX_KIND_ENTITY.get(("tipo", entity), [])

    if not idxs:
        return []

    # Fallback simples caso não tenha matriz de embedding
    if _SCHEMA_EMB_MATRIX is None:
        return (_SCHEMA_TIPOS_BY_ENTITY.get(entity, []) or [])[:top_k]

    q_emb = np.array(_embed_text(query), dtype="float32")
    q_emb /= (np.linalg.norm(q_emb) + 1e-9)

    M = _SCHEMA_EMB_MATRIX[idxs]
    M = M / (np.linalg.norm(M, axis=1, keepdims=True) + 1e-9)
    sims = M @ q_emb

    k_real = min(top_k, sims.shape[0])
    top_idx_local = np.argpartition(-sims, k_real - 1)[:k_real]
    sorted_local = top_idx_local[np.argsort(-sims[top_idx_local])]

    return [_SCHEMA_CHUNKS[idxs[i]] for i in sorted_local]


def ensure_keyword_tipos(tipos, keyword_fields):
    """Garante inclusão de tipos que tenham campos alinhados com palavras-chave."""
    chosen = {t.get("tipo_codigo") for t in (tipos or [])}
    all_t = _SCHEMA_TIPOS_BY_ENTITY.get("AtividadeExec", []) or []

    for t in all_t:
        code = t.get("tipo_codigo")
        if code in chosen:
            continue

        campos = (t.get("campos_exclusivos") or []) + (t.get("campos_comuns") or [])
        if any(c in keyword_fields for c in campos):
            tipos.append(t)
            chosen.add(code)

    return tipos


# --------------------------------------------------------
# 1.6 BUILD CONTEXTO RAG (REFEITO: SCHEMA-FIRST)
# --------------------------------------------------------
def build_rag_context(aischema, question):
    _ensure_schema_index_loaded()

    entities = (aischema.get("targets", {}) or {}).get("entities") or []

    # ------------------------------
    # Fallback antigo (quando IA1 não fornece targets)
    # ------------------------------
    if not entities:
        chunks = search_schema_chunks(query=question, allowed_entities=None, top_k=60)
        if not chunks:
            return "Nenhum contexto encontrado."

        grouped = {}
        global_rels = set()

        for ch in chunks:
            ent = ch.get("entity", "Geral")
            kind = ch.get("kind")

            if kind in ["entity", "field", "tipo"]:
                g = grouped.setdefault(ent, {"descr": "", "fields": [], "tipos": []})
                if kind == "entity":
                    if not g["descr"]:
                        g["descr"] = ch.get("descr", "")
                elif kind == "field":
                    names = [f["name"] for f in g["fields"]]
                    if ch.get("name") not in names:
                        g["fields"].append(ch)
                elif kind == "tipo":
                    codes = [t.get("tipo_codigo") for t in g["tipos"]]
                    if ch.get("tipo_codigo") not in codes:
                        g["tipos"].append(ch)

            elif kind == "rel":
                rel_str = f"({ch.get('de')})-[:{ch.get('tipo_rel')}]->({ch.get('para')})"
                global_rels.add(rel_str)

        lines = []
        lines.append("### ESTRUTURA (Backbone):")
        lines.append("   - (Municipio)-[:TEM_DADO_NO_DIA]->(Dia)")
        lines.append("")

        for ent, info in grouped.items():
            lines.append(f"### Entidade: {ent}")
            if info["descr"]:
                lines.append(f"   DESCRIÇÃO: {_truncate(info['descr'], 200)}")

            if info["fields"]:
                lines.append("   PROPRIEDADES:")
                for f in sorted(info["fields"], key=lambda x: x["name"]):
                    tipo = f"({f.get('tipo','str')})"
                    desc = f" - {_truncate(f.get('descr',''))}"
                    lines.append(f"     - {f['name']} {tipo}{desc}")

            if ent == "AtividadeExec" and info["tipos"]:
                lines.append("   TIPOS SISAWEB (Use: WHERE ae.sisaweb_tipo = ID):")

                def get_code(t):
                    try:
                        return int(t.get("tipo_codigo", 0))
                    except:
                        return 999

                for t in sorted(info["tipos"], key=get_code):
                    tid = t.get("tipo_codigo")
                    name = t.get("name", f"tipo {tid}")
                    desc = _truncate(t.get("descr", ""), 150)
                    campos = t.get("campos_exclusivos", [])
                    lines.append(f"     - {name} -> sisaweb_tipo: {tid}")
                    lines.append(f"       Desc: {desc}")
                    if campos:
                        lines.append(f"       CAMPOS PARA SOMA: {', '.join(campos)}")

            lines.append("")

        if global_rels:
            lines.append("### RELACIONAMENTOS (Conexões Válidas):")
            for r in sorted(list(global_rels)):
                lines.append(f"   - {r}")

        return "\n".join(lines)

    # ------------------------------
    # NOVO MODO: SCHEMA-FIRST guiado pelo PLANO
    # ------------------------------
    lines = []

    # Injeção Manual do Backbone (Segurança)
    lines.append("### ESTRUTURA (Backbone):")
    lines.append("   - (Municipio)-[:TEM_DADO_NO_DIA]->(Dia)")
    lines.append("")

    # 1) Para cada entidade do plano, trazer DESCRIÇÃO + TODOS OS CAMPOS
    for ent in entities:
        lines.append(f"### Entidade: {ent}")

        descr = _SCHEMA_ENTITY_DESCR.get(ent, "")
        if descr:
            lines.append(f"   DESCRIÇÃO: {_truncate(descr, 240)}")

        fields = _SCHEMA_FIELDS_BY_ENTITY.get(ent, []) or []
        if fields:
            lines.append("   PROPRIEDADES:")
            for f in sorted(fields, key=lambda x: x.get("name", "")):
                fname = f.get("name")
                ftype = f.get("tipo", "str")
                fdesc = _truncate(f.get("descr", ""), 200)
                lines.append(f"     - {fname} ({ftype}) - {fdesc}")

        # 2) AtividadeExec -> selecionar melhores TIPOS p/ a pergunta
        if ent == "AtividadeExec":
            tipos_ranked = rank_tipos("AtividadeExec", question, top_k=5)

            # reforço por palavras-chave comuns
            qlow = (question or "").lower()

            # nebulização
            if "nebul" in qlow or "ubv" in qlow:
                tipos_ranked = ensure_keyword_tipos(
                    tipos_ranked,
                    keyword_fields={"nebulizacao", "nebul", "neb"},
                )

            # larvária / liraa / adl (exemplos de reforço leve)
            if "larv" in qlow or "liraa" in qlow or "adl" in qlow:
                tipos_ranked = ensure_keyword_tipos(
                    tipos_ranked,
                    keyword_fields={"ib_larva", "ip_larva", "im_larva", "rec_larva"},
                )

            if tipos_ranked:
                lines.append("   TIPOS SISAWEB (Use: WHERE ae.sisaweb_tipo = ID):")

                def get_code(t):
                    try:
                        return int(t.get("tipo_codigo", 0))
                    except:
                        return 999

                # ordena pelos códigos
                for t in sorted(tipos_ranked, key=get_code):
                    tid = t.get("tipo_codigo")
                    name = t.get("name", f"tipo {tid}")
                    desc = _truncate(t.get("descr", ""), 220)

                    campos_ex = t.get("campos_exclusivos") or []
                    campos_co = t.get("campos_comuns") or []

                    campos_all = []
                    for c in (campos_ex + campos_co):
                        if c not in campos_all:
                            campos_all.append(c)

                    lines.append(f"     - {name} -> sisaweb_tipo: {tid}")
                    lines.append(f"       Desc: {desc}")
                    if campos_all:
                        lines.append(f"       CAMPOS DO TIPO: {', '.join(campos_all)}")

        lines.append("")

    # 3) Todas as relações EXISTENTES entre as entidades do JSON
    rels = get_relations_among_entities(entities)
    if rels:
        lines.append("### RELACIONAMENTOS (Entre Entidades do PLANO):")
        for ch in sorted(
            rels, key=lambda r: (r.get("de", ""), r.get("tipo_rel", ""), r.get("para", ""))
        ):
            lines.append(f"   - ({ch.get('de')})-[:{ch.get('tipo_rel')}]->({ch.get('para')})")

    return "\n".join(lines)


# --------------------------------------------------------
# 2. OLLAMA IA 2 (CYPHER)
# --------------------------------------------------------
def _warm_up_cypher():
    global _cypher_warmed_up
    if _cypher_warmed_up:
        return
    try:
        ollama.chat(model=MODEL_NAME_CYPHER, messages=[{"role": "user", "content": "ping"}])
    except:
        pass
    _cypher_warmed_up = True


def clean_cypher_output(text):
    match = re.search(r"```(?:cypher)?\s*(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()
    match = re.search(r"(MATCH|WITH|CALL)\s.*", text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(0).strip()
    return text.strip()


def cypher_llm_invoke(prompt: str) -> str:
    with _cypher_lock:
        _warm_up_cypher()
        resp = ollama.chat(
            model=MODEL_NAME_CYPHER,
            messages=[{"role": "user", "content": prompt}],
            options={"temperature": 0.0, "num_ctx": 8192},
            stream=False,
        )
    return clean_cypher_output(resp["message"]["content"])


def build_cypher_prompt(question: str, aischema: dict) -> str:
    rag_context = build_rag_context(aischema, question)
    aischema_json = json.dumps(aischema, ensure_ascii=False, indent=2)
    prompt = f"""
PERGUNTA: "{question}"

PLANO (JSON):
{aischema_json}

CONTEXTO DO BANCO (RAG):
{rag_context}
""".strip()
    #print(prompt)
    return prompt


# --------------------------------------------------------
# 3. NEO4J EXECUÇÃO
# --------------------------------------------------------
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))


def run_cypher_in_neo4j(cypher: str):
    if not cypher or "MATCH" not in cypher.upper():
        return []
    if any(x in cypher.upper() for x in ["DELETE", "DETACH", "CREATE", "MERGE", "SET"]):
        return [{"error": "Escrita bloqueada."}]
    with driver.session(database=NEO4J_DATABASE) as session:
        return session.run(cypher, timeout=30).data()


# --------------------------------------------------------
# 4. PIPELINE
# --------------------------------------------------------
# OBS: você não mostrou a implementação da IA1.
# Mantive a chamada como estava no seu código.
# A função abaixo deve existir no seu projeto:
#   - get_ai_schema_dict(question) -> dict
#
# Ela deve retornar algo como o seu PLANO (JSON).
# --------------------------------------------------------
def ask_graph_with_two_ais(question: str) -> dict:
    timers = {}
    t0 = time.time()
    try:
        aischema = get_ai_schema_dict(question)  # <- sua IA1
        timers["ia1"] = aischema.pop("_generation_time", time.time() - t0)
    except Exception as e:
        return {"error": str(e)}

    if not aischema.get("needs_query", True):
        return {
            "question": question,
            "aischema": aischema,
            "cypher": "N/A",
            "result": [],
            "timers": timers,
        }

    t0 = time.time()
    prompt = build_cypher_prompt(question, aischema)
    cypher = cypher_llm_invoke(prompt)
    timers["ia2"] = time.time() - t0

    t0 = time.time()
    try:
        result = run_cypher_in_neo4j(cypher)
        err = None
    except Exception as e:
        result = []
        err = str(e)
    timers["neo4j"] = time.time() - t0

    return {
        "question": question,
        "aischema": aischema,
        "cypher": cypher,
        "result": result,
        "neo4j_error": err,
        "timers": timers,
    }


In [3]:
# =========================
# CÉLULA 3 — DECISÃO RAG (IA2 + pergunta) + RESPOSTA FINAL (IA3)
# Ollama LOCAL + Chroma (Cloud)
# =========================

import os
import json
import re
import chromadb
import ollama

# ---------- Config ----------
GEN_MODEL = os.getenv("OLLAMA_GEN_MODEL", "llama3.1:8b-instruct-q4_K_M")

# Embeddings para RAG (use o MESMO modelo usado para indexar sua coleção no Chroma)
EMBED_MODEL = os.getenv("OLLAMA_EMBED_MODEL", "mxbai-embed-large")

CHROMA_API_KEY        = os.getenv("CHROMA_API_KEY",        "ck-13V15SvUh23Zc7MXYoio9uoNGHgyJLVNcwJw9ZxYr2Z2")
CHROMA_TENANT         = os.getenv("CHROMA_TENANT",         "c3e00254-1f1b-49fb-8f51-c9fbad3c8d76")
CHROMA_DATABASE        = os.getenv("CHROMA_DATABASE", "arbopedia")
CHROMA_COLLECTION_NAME = os.getenv("CHROMA_COLLECTION_NAME", "arbopedia")


# Conecta no Chroma Cloud
chroma_client = chromadb.CloudClient(
    api_key=CHROMA_API_KEY,
    tenant=CHROMA_TENANT,
    database=CHROMA_DATABASE,
)
rag_collection = chroma_client.get_collection(name=CHROMA_COLLECTION_NAME)

# ---------- Helpers ----------
def _safe_json_loads(s: str) -> dict:

    s = s.strip()
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        # tenta capturar bloco {...}
        m = re.search(r"\{.*\}", s, flags=re.S)
        if not m:
            raise
        return json.loads(m.group(0))

def ollama_embed_one(text: str) -> list[float]:
    if hasattr(ollama, "embeddings"):
        return ollama.embeddings(model=EMBED_MODEL, prompt=text)["embedding"]
    if hasattr(ollama, "embed"):
        out = ollama.embed(model=EMBED_MODEL, input=text)
        # out["embeddings"] é lista de vetores; aqui é 1 texto => pega o primeiro
        return out["embeddings"][0]
    raise RuntimeError("Seu pacote 'ollama' não tem embeddings/embed. Atualize o pacote ollama.")

# ---------- IA Router (decide se precisa RAG) ----------
ROUTER_SYSTEM = """
Você é um roteador de RAG. Sua tarefa:
- Recebe a PERGUNTA do usuário e a RESPOSTA (rascunho) da IA2.
- Decida se é necessário consultar a base (Chroma) para responder melhor.
Use RAG quando:
- a pergunta pede fatos/detalhes específicos do domínio,
- a IA2 parece genérica, incerta, incompleta, ou pode estar errada,
- a pergunta depende de conhecimento interno/da base.
Não use RAG quando:
- for conversa casual, opinião pessoal, criatividade pura,
- a IA2 já responde com precisão e completude.
Responda SOMENTE em JSON com as chaves:
{
  "use_rag": boolean,
  "query_text": string,
  "top_k": integer,
  "reason": string
}
"""

def decide_rag(question: str, ia2_answer: str) -> dict:
    resp = ollama.chat(
        model=GEN_MODEL,
        messages=[
            {"role": "system", "content": ROUTER_SYSTEM.strip()},
            {"role": "user", "content": f"PERGUNTA:\n{question}\n\nIA2:\n{ia2_answer}".strip()},
        ],
        format="json",
        options={"temperature": 0.0},
        stream=False,
    )
    data = _safe_json_loads(resp["message"]["content"])
    # defaults seguros
    data.setdefault("use_rag", False)
    data.setdefault("query_text", question)
    data.setdefault("top_k", 6)
    data.setdefault("reason", "")
    # saneamento
    if not isinstance(data["top_k"], int) or data["top_k"] <= 0:
        data["top_k"] = 6
    if not isinstance(data["query_text"], str) or not data["query_text"].strip():
        data["query_text"] = question
    return data

# ---------- Chroma retrieval ----------
def retrieve_rag(query_text: str, top_k: int = 6) -> list[dict]:
    """
    Tenta query_texts primeiro; se falhar (ex.: collection sem embedding function),
    cai para query_embeddings usando embedding local do Ollama.
    """
    include = ["documents", "metadatas", "ids"]
    # tenta incluir distances se o servidor suportar
    try:
        res = rag_collection.query(
            query_texts=[query_text],
            n_results=top_k,
            include=include + ["distances"],
        )
    except Exception:
        emb = ollama_embed_one(query_text)
        res = rag_collection.query(
            query_embeddings=[emb],
            n_results=top_k,
            include=include + ["distances"],
        )

    ids  = (res.get("ids") or [[]])[0]
    docs = (res.get("documents") or [[]])[0]
    metas = (res.get("metadatas") or [[]])[0]
    dists = (res.get("distances") or [[]])[0]

    chunks = []
    for i, doc in enumerate(docs):
        chunks.append({
            "id": ids[i] if i < len(ids) else None,
            "text": doc or "",
            "meta": metas[i] if i < len(metas) else {},
            "distance": dists[i] if i < len(dists) else None,
        })
    return chunks

def build_rag_context_chroma(chunks: list[dict], max_chars: int = 7000) -> str:
    """
    Monta contexto numerado pra IA3 citar como [RAG1], [RAG2], ...
    """
    out = []
    used = 0
    for i, c in enumerate(chunks, 1):
        text = (c.get("text") or "").strip()
        if not text:
            continue
        meta = c.get("meta") or {}
        title = meta.get("title") or meta.get("doc_title") or meta.get("source") or ""
        dist = c.get("distance")
        header = f"[RAG{i}] id={c.get('id')} title={title} dist={dist}"
        block = f"{header}\n{text}\n"
        if used + len(block) > max_chars:
            break
        out.append(block)
        used += len(block)
    return "\n".join(out).strip()

# ---------- IA3 (resposta final) ----------
ANSWER_SYSTEM = """
Você é um assistente que responde em português de forma clara e objetiva.
Você receberá:
- pergunta do usuário
- resposta preliminar da IA2
- (opcional) contexto recuperado do Chroma (RAG), numerado como [RAG1], [RAG2], ...

Regras:
- Se houver contexto RAG, use-o para corrigir/completar a IA2.
- Se você usar um trecho do contexto, cite o identificador [RAGx].
- Se o contexto não trouxer a informação, diga que não encontrou na base e responda com a melhor aproximação (sem inventar fatos).
"""

def generate_final_answer(question: str, ia2_answer: str, rag_context: str | None) -> str:
    user_prompt = f"""
PERGUNTA:
{question}

IA2 (rascunho):
{ia2_answer}

CONTEXTO RAG (se houver):
{rag_context or "(sem RAG)"}

Agora gere a RESPOSTA FINAL.
""".strip()

    resp = ollama.chat(
        model=GEN_MODEL,
        messages=[
            {"role": "system", "content": ANSWER_SYSTEM.strip()},
            {"role": "user", "content": user_prompt},
        ],
        options={"temperature": 0.2, "num_ctx": 8192},
        stream=False,
    )
    return resp["message"]["content"].strip()

# ---------- Função principal da célula ----------
def ia3_answer_with_optional_rag(pergunta_usuario: str, resposta_ia2: str) -> dict:
    decision = decide_rag(pergunta_usuario, resposta_ia2)

    rag_chunks = []
    rag_context = None

    if bool(decision.get("use_rag")):
        rag_chunks = retrieve_rag(decision.get("query_text", pergunta_usuario), top_k=int(decision.get("top_k", 6)))
        rag_context = build_rag_context_chroma(rag_chunks) if rag_chunks else None

        # se não veio nada, cai pra sem RAG
        if not rag_context:
            decision["use_rag"] = False
            decision["reason"] = (decision.get("reason","") + " | RAG retornou vazio").strip()

    final_answer = generate_final_answer(pergunta_usuario, resposta_ia2, rag_context)

    return {
        "final_answer": final_answer,
        "rag_decision": decision,
        "rag_chunks": rag_chunks,
    }

In [4]:

TESTES_STRESS = [
    # =========================================================
    # 1) SIMPLES — CASOS / AGRAVOS (sempre needs_query=True)
    # =========================================================
    {
        "tag": "stress1/simples_casos_01",
        "q": "Qual foi o total de casos de dengue em São José do Rio Preto no ano de 2022?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_02",
        "q": "Como evoluíram, mês a mês, os casos de dengue em São José do Rio Preto desde 2020 até 2024?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_03",
        "q": "Mostre os casos diários de chikungunya em Santos entre 01/02/2023 e 28/02/2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_04",
        "q": "Quantos casos de zika foram registrados em São José do Rio Preto nos últimos 90 dias?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_05",
        "q": "Exiba o número total de casos notificados de dengue em todo o estado de São Paulo em 2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_06",
        "q": "Como foi a série histórica anual de casos de dengue em Ribeirão Preto desde 2020?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_07",
        "q": "Quais foram os casos mensais de chikungunya em São José do Rio Preto em 2021?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_08",
        "q": "Mostre o total de casos de dengue com óbito em São José dos Campos entre 2022 e 2024.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_09",
        "q": "Quantos casos de dengue foram notificados em Sorocaba ontem?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress1/simples_casos_10",
        "q": "Qual é o total acumulado de casos de dengue em Santo André desde o início da base de dados?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 2) METEOROLOGIA PURA (Meteo + Dia, sem agravo)
    # =========================================================
    {
        "tag": "stress2/meteo_01",
        "q": "Mostre a média mensal da temperatura do ar em Campinas em 2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_02",
        "q": "Como variou a temperatura máxima diária em Ribeirão Preto durante o verão de 2022, de dezembro de 2021 a março de 2022?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_03",
        "q": "Exiba a série diária de chuva acumulada em Santos entre 01/01/2024 e 31/03/2024.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_04",
        "q": "Qual foi a umidade relativa média do ar em São José dos Campos nos últimos 30 dias?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_05",
        "q": "Compare a velocidade média do vento ao longo de 2021 em São Paulo e Guarulhos.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_06",
        "q": "Mostre a radiação solar diária em São José do Rio Preto no mês de agosto de 2020.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_07",
        "q": "Como evoluiu a temperatura mínima em Sorocaba de 2020 a 2024, agregada por mês?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_08",
        "q": "Quais foram os dias de 2023 com maior amplitude térmica em Guarujá?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_09",
        "q": "Exiba a precipitação total mensal em todo o estado de São Paulo entre 2020 e 2024.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress2/meteo_10",
        "q": "Nos últimos 7 dias, como se comportaram a temperatura média e a umidade relativa do ar em Bauru?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 3) ATIVIDADES DE CAMPO / SISA (sem citar códigos)
    # =========================================================
    {
        "tag": "stress3/atividades_01",
        "q": "Quantas visitas domiciliares para pesquisa de focos do mosquito foram realizadas em São Paulo no último semestre?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_02",
        "q": "Mostre a quantidade mensal de ações de educação em saúde relacionadas ao controle do Aedes em Campinas em 2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_03",
        "q": "Como evoluiu o número de mutirões ou arrastões para retirada de criadouros em Santos desde 2020?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_04",
        "q": "No ano de 2022, quantos levantamentos de índices de infestação do tipo LIRAa foram feitos em Guarulhos, por mês?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_05",
        "q": "Nos últimos 60 dias, quantas ações de nebulização ou bloqueio de transmissão foram executadas em Sorocaba?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_06",
        "q": "Compare o volume mensal de inspeções em pontos estratégicos, como ferro-velhos e borracharias, entre São José dos Campos e Taubaté em 2021.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_07",
        "q": "Mostre a série trimestral de monitoramento de ovitrampas em Ribeirão Preto de 2020 a 2024.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_08",
        "q": "Quantas inspeções de depósitos e recipientes foram registradas em Diadema no ano passado?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_09",
        "q": "No último ano, qual foi o total de levantamentos detalhados de recipientes por setor censitário realizados em São Bernardo do Campo?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress3/atividades_10",
        "q": "Entre 2021 e 2023, como evoluiu o número de visitas de bloqueio pós-caso suspeito em Osasco, agregando por mês?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 4) NOTIFICAÇÕES / PERFIL DEMOGRÁFICO
    # =========================================================
    {
        "tag": "stress4/notificacoes_01",
        "q": "Nos últimos 90 dias, como se distribuem as notificações de dengue por faixa etária em Campinas?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_02",
        "q": "Em 2023, qual foi a proporção de casos de dengue em gestantes em São José dos Campos?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_03",
        "q": "Mostre, para 2022, a distribuição de casos de dengue por sexo em Guarulhos.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_04",
        "q": "No ano de 2021, quantas notificações de dengue em crianças menores de 5 anos foram registradas em Santos?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_05",
        "q": "Entre 2020 e 2024, como evoluiu o número de internações por dengue em São Paulo, mês a mês?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_06",
        "q": "Compare, em 2023, a proporção de casos de dengue com sinais de alarme entre Ribeirão Preto e São José do Rio Preto.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_07",
        "q": "Mostre as notificações de dengue com óbito em Campinas no período de 01/01/2022 a 31/12/2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_08",
        "q": "Nos últimos 30 dias, quais foram as unidades de saúde de Guarulhos que mais notificaram casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_09",
        "q": "Em 2020, qual foi a distribuição de casos de dengue por escolaridade em Sorocaba?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress4/notificacoes_10",
        "q": "Entre 2021 e 2023, como se comportou a proporção de casos graves de dengue em Santo André?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 5) CORRELAÇÃO METEO × CASOS
    # =========================================================
    {
        "tag": "stress5/meteo_casos_01",
        "q": "Em Campinas, existe associação entre os períodos mais chuvosos e o aumento de casos de dengue entre 2021 e 2023?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_02",
        "q": "Compare, em Santos, a evolução mensal da chuva e dos casos de dengue em 2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_03",
        "q": "Nos últimos 12 meses, em Guarulhos, as semanas com maior temperatura média tiveram mais casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_04",
        "q": "Entre 2020 e 2024, como se relacionam a umidade relativa do ar e os casos mensais de chikungunya em São José dos Campos?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_05",
        "q": "Em Ribeirão Preto, no verão de 2022, houve coincidência entre picos de temperatura máxima e picos de casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_06",
        "q": "No estado de São Paulo como um todo, existe padrão sazonal entre a estação chuvosa e os casos de dengue ao longo dos anos da base?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_07",
        "q": "Em Sorocaba, entre 2022 e 2024, como se comporta a relação entre dias muito quentes e o número de casos notificados de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_08",
        "q": "Compare, em Campinas e Hortolândia, a correlação mensal entre chuva acumulada e casos de dengue em 2021.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_09",
        "q": "Nos últimos 6 meses, em São Paulo, a queda de temperatura se refletiu em redução de casos de chikungunya?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress5/meteo_casos_10",
        "q": "Em Santos, entre 2020 e 2024, como variam os casos de dengue em relação à umidade relativa e à chuva, considerando agregação mensal?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 6) CORRELAÇÃO ATIVIDADES × CASOS
    # =========================================================
    {
        "tag": "stress6/ativ_casos_01",
        "q": "Em São Paulo, nos últimos 12 meses, houve redução de casos de dengue após intensificação das visitas domiciliares?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_02",
        "q": "Compare, em 2023, para Campinas, os meses com mais mutirões de retirada de criadouros e os meses com mais casos de dengue.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_03",
        "q": "Em Guarulhos, entre 2021 e 2023, existe relação entre o número de ações de nebulização e o número de casos de dengue notificados?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_04",
        "q": "Nos últimos 2 anos, em Santos, como a frequência de levantamentos LIRAa se relaciona com a variação dos casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_05",
        "q": "Considerando Sorocaba em 2022, os meses com mais inspeções em pontos estratégicos tiveram menos casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_06",
        "q": "Compare, em Ribeirão Preto, de 2020 a 2024, a evolução dos casos de dengue com a quantidade de ações educativas em saúde realizadas.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_07",
        "q": "Em São José dos Campos, no último ano, houve aumento de nebulizações em períodos de maior incidência de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_08",
        "q": "Entre 2020 e 2023, em Diadema, qual a relação entre monitoramento de ovitrampas e a variação dos casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_09",
        "q": "No Guarujá, durante 2021, mutirões de limpeza concentrados em determinados meses antecederam quedas nos casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress6/ativ_casos_10",
        "q": "Em Osasco, de 2022 a 2024, como se relacionam os casos de dengue com o número de visitas de bloqueio realizadas após detecção de casos?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 7) COMPARAÇÃO ENTRE MUNICÍPIOS
    # =========================================================
    {
        "tag": "stress7/compare_muni_01",
        "q": "Compare os casos mensais de dengue em São Paulo, Guarulhos e Osasco em 2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_02",
        "q": "Quais municípios do litoral paulista tiveram mais casos de dengue em 2022: Santos, São Vicente ou Guarujá?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_03",
        "q": "Entre Campinas, Ribeirão Preto e São José do Rio Preto, qual teve mais casos de chikungunya em 2021, por mês?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_04",
        "q": "Compare a proporção de casos de dengue com óbito entre São José dos Campos e Taubaté entre 2020 e 2024.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_05",
        "q": "Nos últimos 6 meses, compare o número de notificações de dengue em Diadema, Santo André e São Bernardo do Campo.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_06",
        "q": "Em 2023, quais foram as diferenças na distribuição por faixa etária dos casos de dengue entre Campinas e Hortolândia?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_07",
        "q": "Compare a evolução anual de casos de dengue em municípios de porte semelhante, como Bauru e Marília, de 2020 a 2024.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_08",
        "q": "Entre Sorocaba e Jundiaí, qual município apresentou picos de casos de dengue mais precoces em 2022?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_09",
        "q": "Compare, em 2021, a quantidade de visitas domiciliares para controle do Aedes em São Paulo, Guarulhos e Osasco.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress7/compare_muni_10",
        "q": "Nos últimos 3 meses, quais diferenças existem entre Santos e Praia Grande quanto ao volume diário de casos de dengue?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 8) COMPARAÇÃO ENTRE PERÍODOS
    # =========================================================
    {
        "tag": "stress8/compare_periodos_01",
        "q": "Compare o número médio mensal de casos de dengue em São Paulo antes e depois de 2022.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_02",
        "q": "Em Campinas, compare o verão de 2020/2021 com o verão de 2023/2024 em termos de casos de dengue.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_03",
        "q": "Em Santos, como os casos de dengue de 2020 se comparam aos de 2022, mês a mês?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_04",
        "q": "Em Guarulhos, houve diferença entre o primeiro semestre de 2023 e o primeiro semestre de 2024 na quantidade de casos de chikungunya?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_05",
        "q": "Compare, em Ribeirão Preto, os casos mensais de dengue entre o período de 01/01/2021 a 31/12/2021 e o período de 01/01/2023 a 31/12/2023.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_06",
        "q": "Em São José dos Campos, os últimos 90 dias tiveram mais casos de dengue do que os 90 dias anteriores?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_07",
        "q": "Em Sorocaba, compare o mês passado com os últimos 12 meses no que se refere a notificações de dengue.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_08",
        "q": "Em Santo André, houve mudança no perfil etário dos casos de dengue entre 2020-2021 e 2022-2023?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_09",
        "q": "No Guarujá, compare a intensidade das ações de visita domiciliar antes e depois do início de 2022.",
        "expect_needs_query": True,
    },
    {
        "tag": "stress8/compare_periodos_10",
        "q": "Em Campinas, o inverno de 2021 teve menos casos de dengue que o inverno de 2023?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 9) RANKINGS / Picos / Outliers
    # =========================================================
    {
        "tag": "stress9/ranking_01",
        "q": "Nos últimos 6 meses, quais foram os cinco municípios com maior número de casos de dengue no estado de São Paulo?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_02",
        "q": "Entre 2020 e 2024, quais municípios apresentaram os maiores picos mensais de casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_03",
        "q": "No último ano, entre São Paulo, Guarulhos, Osasco, Santo André, São Bernardo do Campo e Diadema, quais mais notificaram casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_04",
        "q": "Quais municípios tiveram o maior número de notificações de dengue com óbito em 2023?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_05",
        "q": "Nos últimos 90 dias, entre Santos, São Vicente, Guarujá, Praia Grande e Cubatão, quais concentraram mais casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_06",
        "q": "Em 2022, quais municípios apresentaram a maior taxa de crescimento mensal de casos de dengue em algum momento do ano?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_07",
        "q": "Entre 2020 e 2024, quais municípios mantiveram consistentemente baixos números de casos de dengue em comparação com a média do estado?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_08",
        "q": "No último trimestre, quais municípios médios do interior, como Bauru, Marília, Presidente Prudente e São José do Rio Preto, tiveram mais casos de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_09",
        "q": "Em 2021, quais municípios entre Campinas, Ribeirão Preto e Sorocaba tiveram, em média, mais casos mensais de dengue?",
        "expect_needs_query": True,
    },
    {
        "tag": "stress9/ranking_10",
        "q": "Nos últimos 30 dias, quais foram os dez municípios com maior número médio diário de casos de dengue?",
        "expect_needs_query": True,
    },

    # =========================================================
    # 10) CONCEITUAIS / RAG (needs_query=False)
    # =========================================================
    {
        "tag": "stress10/doc_01",
        "q": "Explique, de forma resumida, como as atividades de visita domiciliar contribuem para o controle do Aedes aegypti.",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_02",
        "q": "Quais são as principais diferenças entre dengue, zika e chikungunya em termos de modo de transmissão e sintomas gerais?",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_03",
        "q": "Descreva o papel das ovitrampas na vigilância do mosquito Aedes aegypti.",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_04",
        "q": "Como um levantamento LIRAa costuma ser planejado e utilizado na gestão do controle vetorial?",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_05",
        "q": "Quais são as etapas típicas de resposta de um serviço de saúde municipal diante de um aumento repentino de casos suspeitos de dengue?",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_06",
        "q": "Explique, em linguagem simples, como os dados meteorológicos podem apoiar o planejamento das ações de controle do Aedes.",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_07",
        "q": "Que tipos de informações costumam constar em uma ficha de notificação individual de dengue?",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_08",
        "q": "Descreva boas práticas para comunicação de risco à população durante períodos de alta transmissão de dengue.",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_09",
        "q": "Explique o que se entende por caso grave de dengue e quais sinais de alerta exigem maior atenção na rede de saúde.",
        "expect_needs_query": False,
    },
    {
        "tag": "stress10/doc_10",
        "q": "Como um grafo de dados pode ajudar a integrar informações de municípios, dias, atividades de campo, notificações e variáveis climáticas em uma mesma visão?",
        "expect_needs_query": False,
    },
]

TESTES_BASICOS = [

    {
        "tag": "simples/atividade",
        "q": "Atividades de nebulização no último mês em São José do Rio Preto",
        "expect_needs_query": True,
    },
    {
        "tag": "simples/meteo",
        "q": "Temperatura do ultimo ano em São José do Rio Preto",
        "expect_needs_query": True,
    },
    {
        "tag": "simples/ano",
        "q": "Casos de dengue em São José do Rio Preto no ano de 2024",
        "expect_needs_query": True,
    },
    {
        "tag": "erros/ortografia",
        "q": "cazo de dengi em São José do Rio Preto",
        "expect_needs_query": True,
    },
    {
        "tag": "tempo/7d",
        "q": "Notificações de dengue nos últimos 7 dias em São José do Rio Preto",
        "expect_needs_query": True,
    },
    {
        "tag": "tempo/range",
        "q": "Temperatura e umidade de 01/02/2023 a 15/03/2023 em Campinas",
        "expect_needs_query": True,
    },
    {
        "tag": "tempo/ultimo_trimestre",
        "q": "Comparar casos por mês no último trimestre em São Paulo e Campinas",
        "expect_needs_query": True,
    },
    {
        "tag": "tempo/mes_passado",
        "q": "Casos de dengue no mês passado em Santos",
        "expect_needs_query": True,
    },
]

TESTES_COMPLEXOS = [

    # 2) Temperatura x nebulização (atividade 3) em um município, agregando por semana
    {
        "tag": "correlacao/temp_nebulizacao_semana",
        "q": (
            "No último ano, em Campinas, as semanas com temperatura média diária mais alta "
            "tiveram mais ações de nebulização ou bloqueio (atividade 3) associadas à dengue? "
            "Use a temperatura média T2M e agregue os dados por semana."
        ),
        "expect_needs_query": True,
    },

    # 3) Comparar evolução de vários agravos em 2 municípios
    {
        "tag": "compare/multi_agravo_multi_muni",
        "q": (
            "Compare a evolução mensal de casos de dengue, zika e chikungunya em Ribeirão Preto "
            "e São José do Rio Preto durante todo o ano de 2022, mostrando os três agravos lado a lado."
        ),
        "expect_needs_query": True,
    },

    # 4) Notificações individuais x casos agregados com óbito, mesmo município
    {
        "tag": "compare/notif_vs_casos_obitos",
        "q": (
            "Entre 01/01/2023 e 31/12/2023, em Sorocaba, como se comparam o número total de "
            "notificações individuais de dengue e os casos agregados com óbito, mês a mês?"
        ),
        "expect_needs_query": True,
    },

    # 5) Volume de visitas domiciliares (atividade 1) x variação de casos de dengue em 3 municípios
    {
        "tag": "correlacao/visita_domiciliar_casos",
        "q": (
            "Nas cidades de São Paulo, Guarulhos e Osasco, existe associação entre o volume de "
            "visitas domiciliares/pesquisa larvária (atividade 1) e a variação mensal de casos "
            "de dengue em 2023?"
        ),
        "expect_needs_query": True,
    },

    # 6) Meteo com mais de uma variável x agravo específico
    {
        "tag": "correlacao/meteo_multi_var_chik",
        "q": (
            "Em Campinas, qual é a relação entre a média mensal de temperatura (T2M) e umidade "
            "relativa (RH2M) e os casos de chikungunya em 2021? "
            "Destaque os meses com picos de casos."
        ),
        "expect_needs_query": True,
    },

    # 7) Encontrar top municípios por notificações no último semestre (+ evolução de casos)
    {
        "tag": "ranking/notif_ultimo_semestre",
        "q": (
            "No último semestre, quais foram os três municípios com maior número de notificações "
            "de dengue no estado de São Paulo e como evoluíram os casos agregados nesses municípios "
            "no mesmo período?"
        ),
        "expect_needs_query": True,
    },

    # 8) Série longa de LIRAa (atividade 6) x casos de dengue, trimestral
    {
        "tag": "serie/liraa_casos_trimestral",
        "q": (
            "Mostre a série histórica trimestral de LIRAa ou levantamentos amostrais "
            "(atividade 6) e dos casos de dengue em Santo André de 01/01/2020 a 31/12/2024, "
            "para avaliar tendências ao longo dos anos."
        ),
        "expect_needs_query": True,
    },

    # 9) Comparação de dois períodos diferentes no mesmo município (exige interpretação de períodos)
    {
        "tag": "compare/dois_periodos_mesmo_muni",
        "q": (
            "Compare a média mensal de casos de dengue em São Paulo no verão 2020/2021 "
            "(de 01/12/2020 a 31/03/2021) com o verão 2022/2023 "
            "(de 01/12/2022 a 31/03/2023)."
        ),
        "expect_needs_query": True,
    },

    # 10) Perfil de notificações + atividades de bloqueio (atividade 3) no mesmo período
    {
        "tag": "perfil/notif_bloqueio_90d",
        "q": (
            "Em Guarulhos, nos últimos 90 dias, como se distribuem as notificações de dengue por "
            "faixa etária e sexo, e houve intensificação de atividades de bloqueio/nebulização "
            "(atividade 3) nas áreas com mais notificações?"
        ),
        "expect_needs_query": True,
    },

    # 11) Casos graves/hospitalização x total de casos
    {
        "tag": "gravidade/hosp_obito_vs_total",
        "q": (
            "Entre 2022 e 2024, em São José dos Campos, qual foi a evolução mensal dos casos de "
            "dengue com hospitalização ou óbito em comparação com o total de casos notificados?"
        ),
        "expect_needs_query": True,
    },

    # 12) Ovitrampas (atividade 8) x chuva (PRECTOTCORR)
    {
        "tag": "correlacao/ovitrampas_chuva",
        "q": (
            "No último ano, há correlação entre a quantidade de ovitrampas positivas para Aedes aegypti "
            "(atividade 8) e a precipitação total corrigida (PRECTOTCORR) em Campinas?"
        ),
        "expect_needs_query": True,
    },

    # 13) Controle vetorial combinado (atividades 1 e 6) x casos de dengue em 2 municípios
    {
        "tag": "correlacao/controle_combinado_casos",
        "q": (
            "Compare, para Santos e Guarujá, a relação entre os casos mensais de dengue e as ações "
            "de visita domiciliar/pesquisa larvária (atividade 1) e LIRAa/levantamento amostral "
            "(atividade 6) de 2021 a 2023."
        ),
        "expect_needs_query": True,
    },

    # 14) Atraso entre sintomas e notificação x gravidade/óbito em um único município
    {
        "tag": "correlacao/atraso_notificacao_gravidade",
        "q": (
            "Em São Paulo, em 2023, as unidades de saúde com maior atraso entre a data dos primeiros "
            "sintomas e a data de notificação tiveram maior proporção de casos graves ou óbitos "
            "por dengue?"
        ),
        "expect_needs_query": True,
    },

    # 15) Full combo: casos + meteo + atividades + notificações graves em 3 municípios, 2 anos
    {
        "tag": "full/combo_casos_meteo_ativ_notif",
        "q": (
            "Considerando o período de 01/01/2022 a 31/12/2023, para São Paulo, Campinas e Sorocaba, "
            "faça uma análise comparando: (a) casos mensais de dengue, "
            "(b) precipitação total corrigida (PRECTOTCORR), "
            "(c) número de ações de nebulização ou bloqueio (atividade 3) e "
            "(d) número de notificações com hospitalização ou óbito, "
            "destacando meses em que todos esses indicadores estiveram altos ao mesmo tempo."
        ),
        "expect_needs_query": True,
    },
]

TESTES_CONCEITUAIS = [
    {
        "tag": "doc/prevencao_dengue",
        "q": (
            "Quais são as principais estratégias de prevenção da dengue recomendadas para áreas urbanas, "
            "considerando ações da população e do serviço de saúde?"
        ),
        "expect_needs_query": False,
    },
    {
        "tag": "doc/conduta_clinica_inicial",
        "q": (
            "Como deve ser a conduta clínica inicial para um paciente com suspeita de dengue clássica, "
            "sem sinais de alarme, segundo os protocolos de manejo clínico?"
        ),
        "expect_needs_query": False,
    },
    {
        "tag": "doc/definicao_meteo",
        "q": (
            "Explique, em linguagem simples, o que representa o nó Meteo no grafo "
            "e quais são os principais tipos de variáveis meteorológicas disponíveis."
        ),
        "expect_needs_query": False,
    },
    {
        "tag": "doc/definicao_liraa",
        "q": (
            "O que é o LIRAa ou LIA no contexto das atividades de controle vetorial "
            "e como esse tipo de atividade aparece estruturado na base de dados?"
        ),
        "expect_needs_query": False,
    },
    {
        "tag": "doc/estrutura_grafo",
        "q": (
            "Descreva de forma resumida como o grafo está organizado para representar "
            "municípios, dias, atividades, casos, notificações e agravos, sem trazer números, "
            "apenas explicando a estrutura e os relacionamentos principais."
        ),
        "expect_needs_query": False,
    },
]

ALL_TESTES = TESTES_BASICOS + TESTES_COMPLEXOS + TESTES_CONCEITUAIS + TESTES_STRESS

In [ ]:
import time
import json

def run_tests_pipeline_layout_final(
    tests, 
    pipeline_fn, 
    titulo="TESTES PIPELINE (LAYOUT COMPLETO)", 
    mostrar_apenas_erros=False
):
    resultados = []
    t_start_suite = time.time()
    
    print(f"\n===== {titulo} =====")
    print(f"Total de testes: {len(tests)}")

    for i, item in enumerate(tests, 1):
        q = item["q"]
        tag = item.get("tag", f"test_{i:03d}")
        status = "ok"
        
        try:
            # --- EXECUÇÃO ---
            resp = pipeline_fn(q)
            
            # Extração de dados com defaults seguros
            aischema = resp.get("aischema") or {}
            cypher = resp.get("cypher") or ""
            neo4j_result = resp.get("result") or []
            neo4j_error = resp.get("neo4j_error")
            
            # Timers (com fallback se vier vazio)
            timers = resp.get("timers") or {}
            t_ia1 = timers.get("ia1", timers.get("ia1_schema", 0.0))
            t_ia2 = timers.get("ia2", timers.get("ia2_cypher", 0.0))
            t_neo = timers.get("neo4j", timers.get("neo4j_exec", 0.0))
            t_total = t_ia1 + t_ia2 + t_neo

            # Validação rápida para status
            if not aischema: status = "erro_ia1"
            elif neo4j_error: status = "erro_neo4j"
            
            # --- LAYOUT DE EXIBIÇÃO ---
            deve_mostrar = (status != "ok") or (not mostrar_apenas_erros)
            
            if deve_mostrar:
                icon = "✅" if status == "ok" else "❌"
                # Alerta se query vazia (mas esperada) ou resultado vazio
                if status == "ok" and aischema.get("needs_query") and not neo4j_result: 
                    icon = "⚠️"

                print("\n" + "=" * 80)
                # LINHA 1: Pergunta e Tempos
                print(f"[{i:03d}] {icon} PERGUNTA: {q}")
                print(f"      (IA1: {t_ia1:.2f}s + IA2: {t_ia2:.2f}s + Neo4j: {t_neo:.2f}s = TOTAL: {t_total:.2f}s)")
                print("-" * 80)

                # BLOCO 1: Resultado Íntegro da IA 1
                print("🔹 [IA 1] SCHEMA JSON:")
                if aischema:
                    print(json.dumps(aischema, indent=2, ensure_ascii=False))
                else:
                    print("   (Vazio/Erro)")
                print("-" * 40)

                # BLOCO 2: Resultado Íntegro da IA 2 (se houver)
                if aischema.get("needs_query", False):
                    print("🔹 [IA 2] CYPHER QUERY:")
                    if cypher.strip():
                        print(cypher)
                    else:
                        print("   [VAZIO - IA não gerou código ou filtro removeu tudo]")
                    
                    print("-" * 40)

                    # BLOCO 3: Resultado do Banco ou Erro
                    if neo4j_error:
                        print(f"🔥 [NEO4J] ERRO CRÍTICO:")
                        print(neo4j_error)
                    else:
                        qtd = len(neo4j_result)
                        print(f"🔹 [NEO4J] RESULTADO ({qtd} linhas):")
                        if qtd == 0:
                            print("   [Lista Vazia]")
                        else:
                            # Mostra até 5 exemplos
                            for idx, row in enumerate(neo4j_result[:5]):
                                print(f"   {idx+1}. {row}")
                            if qtd > 5:
                                print(f"   ... (mais {qtd-5} registros ocultos)")
                else:
                    print("🔹 [IA 2] N/A (Pergunta Conceitual - Sem Query)")

            resultados.append({"tag": tag, "status": status})

        except Exception as e:
            print(f"\n❌ ERRO FATAL NO TESTE {tag}: {e}")
            resultados.append({"tag": tag, "status": "erro_runner"})

    print("\n" + "=" * 80)
    print(f"TEMPO TOTAL DA SUITE: {time.time() - t_start_suite:.2f}s")
    return resultados

# Executar
if __name__ == "__main__":
    run_tests_pipeline_layout_final(
        ALL_TESTES, 
        ask_graph_with_two_ais, 
        mostrar_apenas_erros=False
    )


===== TESTES — PIPELINE COMPLETO (IA1 + IA2 + Neo4j + IA2_ANSWER + IA3/RAG) =====
Total de testes: 127

❌ ERRO FATAL NO TESTE simples/atividade: Object of type Date is not JSON serializable
